In [ ]:
!pip install langchain

zhipuai_api_key = "41e34b6365edaa50266d9ed13a1f953b.CodlLOLDqAj7UvEK"

!pip install zhipuai

from zhipuai import ZhipuAI
client = ZhipuAI(api_key=zhipuai_api_key) # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
        {"role": "user", "content": "你好"},
        {"role": "assistant", "content": "我是人工智能助手"},
        {"role": "user", "content": "你叫什么名字"},
        {"role": "assistant", "content": "我叫chatGLM"},
        {"role": "user", "content": "你都可以做些什么事"}
        
    ],
)
print(response.choices[0].message)
print(response.choices[0].message.content)


import json
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import List, Optional
from zhipuai import ZhipuAI
from langchain_core.messages.ai import AIMessage
zhipuai_api_key = "1354616636358b562965c51dc1da555d.sBYNC7rIcp21aEoe"

def tool_config_from_file(tool_name, directory="Tool/"):
    """search tool yaml and return json format"""
    for filename in os.listdir(directory):
        if filename.endswith('.yaml') and tool_name in filename:
            file_path = os.path.join(directory, filename)
            with open(file_path, encoding='utf-8') as f:
                return yaml.safe_load(f)
    return None


class ChatGLM4(LLM):
    max_token: int = 8192
    do_sample: bool = True
    temperature: float = 0.8
    top_p = 0.8
    tokenizer: object = None
    model: object = None
    history: List = []
    tool_names: List = []
    has_search: bool = False
    client:object =None
    def __init__(self):
        super().__init__()
        self.client = ZhipuAI(api_key=zhipuai_api_key) 

    @property
    def _llm_type(self) -> str:
        return "ChatGLM4"

    
    def stream(self,prompt:str,history=[]):
        if history is None:
            history=[]
            
        history.append({"role": "user", "content": prompt})
        response = self.client.chat.completions.create(
            model="glm-4",  # 填写需要调用的模型名称
            messages=history,
            stream=True,
        )
        for chunk in response:
            yield chunk.choices[0].delta.content
        
        

    def _tool_history(self, prompt: str):
        ans = []
        tool_prompts = prompt.split(
            "You have access to the following tools:\n\n")[1].split("\n\nUse a json blob")[0].split("\n")

        tool_names = [tool.split(":")[0] for tool in tool_prompts]
        self.tool_names = tool_names
        tools_json = []
        for i, tool in enumerate(tool_names):
            tool_config = tool_config_from_file(tool)
            if tool_config:
                tools_json.append(tool_config)
            else:
                ValueError(
                    f"Tool {tool} config not found! It's description is {tool_prompts[i]}"
                )

        ans.append({
            "role": "system",
            "content": "Answer the following questions as best as you can. You have access to the following tools:",
            "tools": tools_json
        })
        query = f"""{prompt.split("Human: ")[-1].strip()}"""
        return ans, query

    def _extract_observation(self, prompt: str):
        return_json = prompt.split("Observation: ")[-1].split("\nThought:")[0]
        self.history.append({
            "role": "observation",
            "content": return_json
        })
        return

    def _extract_tool(self):
        if len(self.history[-1]["metadata"]) > 0:
            metadata = self.history[-1]["metadata"]
            content = self.history[-1]["content"]
            if "tool_call" in content:
                for tool in self.tool_names:
                    if tool in metadata:
                        input_para = content.split("='")[-1].split("'")[0]
                        action_json = {
                            "action": tool,
                            "action_input": input_para
                        }
                        self.has_search = True
                        return f"""
Action: 
```
{json.dumps(action_json, ensure_ascii=False)}
```"""
        final_answer_json = {
            "action": "Final Answer",
            "action_input": self.history[-1]["content"]
        }
        self.has_search = False
        return f"""
Action: 
```
{json.dumps(final_answer_json, ensure_ascii=False)}
```"""

    def _call(self, prompt: str, history: List = [], stop: Optional[List[str]] = ["<|user|>"]):
        if history is None:
            history=[]
            
        history.append({"role": "user", "content": prompt})
        response = self.client.chat.completions.create(
            model="glm-4",  # 填写需要调用的模型名称
            messages=history,
        )
        
        
        result = response.choices[0].message.content
        print(result)
        #AIMessage(content=result)
        return result

    
llm = ChatGLM4()

from langchain import PromptTemplate
template = """{question}"""
prompt = PromptTemplate(template=template, input_variables=["question"])

from langchain import LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "北京和上海两座城市有什么不同？"

llm_chain.run(question)



from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)


# Requires:
# pip install langchain docarray tiktoken
import sentence_transformers
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
# replace to your model path
# embedding_path = "D:\\ai\\download\\text2vec-large-chinese"
embedding_path="D:\\ai\\bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)
vectorstore = FAISS.from_documents(
    documents=all_splits,
    embedding=embeddings,
)


question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

# Prompt
prompt = PromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | llm | StrOutputParser()

# Run
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
chain.invoke(docs)


# Raptor
from typing import Dict, List, Optional, Tuple
 
import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture
 
RANDOM_SEED = 224  # Fixed seed for reproducibility
 
 
def global_cluster_embeddings(
        embeddings: np.ndarray,
        dim: int,
        n_neighbors: Optional[int] = None,
        metric: str = "cosine",
) -> np.ndarray:
    """
    使用 UMAP 对嵌入执行全局降维。
    参数：
    - embeddings：作为 numpy 数组的输入嵌入。
    - dim：缩减空间的目标维度。
    - n_neighbors：可选；数字每个点要考虑的邻居数。如果未提供，则默认为嵌入数量的平方根。
    - metric：用于 UMAP 的距离度量。
    返回：
    - 减少到指定维度的嵌入的 numpy 数组。
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)
 
 
def local_cluster_embeddings(
        embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    使用 UMAP 对嵌入执行局部降维，通常在全局聚类之后。
    参数：
    - embeddings：作为 numpy 数组的输入嵌入。
    - dim：缩减空间的目标维度。
    - num_neighbors：每个点要考虑的邻居数量。
    - metric：用于 UMAP 的距离度量。
    返回：
    - 减少到指定维度的嵌入的 numpy 数组。
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)
 
 
def get_optimal_clusters(
        embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    使用贝叶斯信息准则 (BIC) 和高斯混合模型确定最佳簇数。
    参数：
    - embeddings：作为 numpy 数组的输入嵌入。
    - max_clusters：要考虑的最大簇数。
    - random_state：可重复性的种子。
    返回：
    - 表示找到的最佳簇数的整数。
    """
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]
 
 
def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    使用高斯聚类嵌入基于概率阈值的混合模型 (GMM)。
    参数：
    - embeddings：作为 numpy 数组的输入嵌入。
    - 阈值：将嵌入分配给簇的概率阈值。
    - random_state：可重复性的种子。
    返回：
    - 包含聚类标签和确定的聚类数量的元组。
    """
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters
 
 
def perform_clustering(
        embeddings: np.ndarray,
        dim: int,
        threshold: float,
) -> List[np.ndarray]:
    """
    通过首先全局降低嵌入的维数，然后使用高斯混合模型进行聚类，最后对嵌入进行聚类在每个全局聚类中执行局部聚类。
    参数：
    - embeddings：作为 numpy 数组的输入嵌入。
    - dim：UMAP 缩减的目标维数。
    - Threshold：在 GMM 中将嵌入分配给聚类的概率阈值。
    返回：
    - numpy 数组列表，其中每个数组包含每个嵌入的簇 ID。
    """
    if len(embeddings) <= dim + 1:
        # Avoid clustering when there's insufficient data
        return [np.array([0]) for _ in range(len(embeddings))]
 
    # Global dimensionality reduction
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # Global clustering
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )
 
    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0
 
    # Iterate through each global cluster to perform local clustering
    for i in range(n_global_clusters):
        # Extract embeddings belonging to the current global cluster
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]
 
        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # Handle small clusters with direct assignment
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # Local dimensionality reduction and clustering
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )
 
        # Assign local cluster IDs, adjusting for total clusters already processed
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )
 
        total_clusters += n_local_clusters
 
    return all_local_clusters
 
 
def embed(texts):
    """
    为文本文档列表生成嵌入。
    此函数假设存在 `embd` 对象，其方法 `embed_documents`
    接受文本列表并返回其嵌入。
    参数：
    - texts：List[str]，要嵌入的文本文档列表。
    返回：
    - numpy.ndarray：嵌入数组
    """
    text_embeddings = embd.embed_documents(texts)
    text_embeddings_np = np.array(text_embeddings)
    return text_embeddings_np
 
 
def embed_cluster_texts(texts):
    """
    嵌入文本列表并对它们进行聚类，返回包含文本的DataFrame 、它们的嵌入和聚类标签。
    该函数将嵌入生成和聚类合并为一个步骤。它假设存在先前定义的“perform_clustering”函数，该函数对嵌入执行聚类。
    参数：
        - texts：List[str]，要处理的文本文档列表。
    返回：
        - pandas.DataFrame：包含原始文本、其嵌入以及分配的簇标签的 DataFrame。
    """
    text_embeddings_np = embed(texts)  # Generate embeddings
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )  # Perform clustering on the embeddings
    df = pd.DataFrame()  # Initialize a DataFrame to store the results
    df["text"] = texts  # Store original texts
    df["embd"] = list(text_embeddings_np)  # Store embeddings as a list in the DataFrame
    df["cluster"] = cluster_labels  # Store cluster labels
    return df
 
 
def fmt_txt(df: pd.DataFrame) -> str:
    """
    将 DataFrame 中的文本文档格式化为单个字符串.
    参数：
    - df：包含要格式化的文本文档的 'text' 列的 DataFrame。
    返回：
    - 所有文本文档均由特定分隔符连接的单个字符串。
    """
    unique_txt = df["text"].tolist()
    return "--- --- \n --- --- ".join(unique_txt)
 
 
def embed_cluster_summarize_texts(
        texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    嵌入、聚类和总结文本列表。此函数首先生成文本的嵌入，
    根据相似性对它们进行聚类，扩展聚类分配以便于处理，然后总结每个聚类内的内容。
    参数：
    - texts：要处理的文本文档列表。
    - level：一个整数参数，可以定义处理的深度或细节。
    返回：
    - 包含两个 DataFrame 的元组：
      1. 第一个 DataFrame (`df_clusters`) 包含原始文本、它们的嵌入和聚类分配。
      2. 第二个 DataFrame (`df_summary`) 包含每个集群的摘要、指定的详细级别以及集群标识符。
    """
 
    # Embed and cluster the texts, resulting in a DataFrame with 'text', 'embd', and 'cluster' columns
    df_clusters = embed_cluster_texts(texts)
 
    # Prepare to expand the DataFrame for easier manipulation of clusters
    expanded_list = []
 
    # Expand DataFrame entries to document-cluster pairings for straightforward processing
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )
 
    # Create a new DataFrame from the expanded list
    expanded_df = pd.DataFrame(expanded_list)
 
    # Retrieve unique cluster identifiers for processing
    all_clusters = expanded_df["cluster"].unique()
 
    print(f"--Generated {len(all_clusters)} clusters--")
 
    # Summarization
    template = """Here is a sub-set of LangChain Expression Langauge doc.
    LangChain Expression Langauge provides a way to compose chain in LangChain.
    Give a detailed summary of the documentation provided.
    Documentation:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()
 
    # Format text within each cluster for summarization
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))
 
    # Create a DataFrame to store summaries with their corresponding cluster and level
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )
 
    return df_clusters, df_summary
 
 
def recursive_embed_cluster_summarize(
        texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    递归地嵌入、聚类和汇总文本，直到指定级别或直到唯一聚类的数量变为 1，存储每个级别的结果。
    参数：
    - texts：List[str]，要处理的文本。
    - level：int，当前递归级别（从1开始）。
    - n_levels：int，最大递归深度。
    返回：
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]，一个字典，其中键是递归级别，值是包含该级别的簇 DataFrame 和摘要 DataFrame 的元组。
    """
    results = {}  # Dictionary to store results at each level
 
    # Perform embedding, clustering, and summarization for the current level
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)
 
    # Store the results of the current level
    results[level] = (df_clusters, df_summary)
 
    # Determine if further recursion is possible and meaningful
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # Use summaries as the input texts for the next level of recursion
        new_texts = df_summary["summaries"].tolist()
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )
 
        # Merge the results from the next level into the current results dictionary
        results.update(next_level_results)
 
    return results


leaf_texts = docs
results = recursive_embed_cluster_summarize(leaf_texts, level=1, n_levels=3)

# Initialize all_texts with leaf_texts
all_texts = leaf_texts.copy()
 
# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)
#Final Summaries extracted
print(all_texts)

vectorstore = Chroma.from_texts(texts=all_texts, embedding=embddinga) 
retriever = vectorstore.as_retriever()

from langchain import hub
from langchain_core.runnables import RunnablePassthrough
 
# Prompt
prompt = hub.pull("rlm/rag-prompt")
 
 
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
 
 
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(prompt)
print(prompt.messages[0].prompt.template)